# Étude sur les facteurs qui déterminent la santé des enfants aux États-Unis

# Importation des libraries 

In [1]:
import s3fs # pour récupérer les fichiers de l'espace de stockage S3
import pandas as pd # pour la gestion des dataframe
import geopandas as gpd # pour la gestion des données géographiques (fichiers .shp)
import matplotlib.pyplot as plt
import script.clean_data as cd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

## Récupération des bases de données

In [2]:
# Maximum 15 secondes

# (depuis le stockage S3 d'un membre du groupe via le dossier public diffusion)
# Stocker les fichiers dans un dossier data/nsch

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

chemin_lecture_nsch = "inacampan/diffusion/Determinants_of_children-s_health/NSCH/"
chemin_ecriture_nsch = "data/nsch/"

# Lecture des fichiers des bases de données
dfs = cd.lecture_fichier_sas(fs, chemin_lecture_nsch, chemin_ecriture_nsch)

# récuperér le guide technique du questionnaire
# ce fichier inclut le nom de la variable, la question qu'elle encode, l'année du questionnaire
# mais également des notes pour la compréhension de la base de données initiale

guide = cd.lecture_fichier_csv(fs, f"{chemin_lecture_nsch}NSCH_Dictionary.csv", f"{chemin_ecriture_nsch}NSCH_Dictionary.csv")

# Element de précaution : GUIDE contient plusieurs index de meme nom
# mais les caractéristiques ne sont pas identiques

In [3]:
# Maximum 2 secondes

# Récuperer les fichiers associés au shapefile - utiles pour les cartes (depuis le stockage S3 d'un membre du groupe)

chemin_lecture_map = "inacampan/diffusion/Determinants_of_children-s_health/Map/"
chemin_ecriture_map = "data/map/"

# lecture du fichier
gdf = cd.lecture_fichier_shapefile(fs, chemin_lecture_map, chemin_ecriture_map)

## Description des données et statistiques descriptives

Conformément au site qui héberge les bases de données NSCH, le questionnaire de présélection (NSCH-S1) déterminait si l’adresse représentait une résidence occupée et s’il y avait des enfants admissibles âgés de 0 à 17 ans vivant à cette adresse échantillonnée.

Le questionnaire thématique ("topical") comprenait des questions détaillées portant sur un enfant sélectionné au hasard dans le ménage. Les ménages recevaient l’un des trois questionnaires thématiques spécifiques à l’âge, en fonction de l’âge de l’enfant échantillonné : 
* NSCH-T1 (ou T1) pour les enfants âgés de 0 à 5 ans,
* NSCH-T2 (ou T2) pour les enfants âgés de 6 à 11 ans, ou
* NSCH-T3 (ou T3) pour les enfants âgés de 12 à 17 ans.

Le type du questionnaire est codé dans la variable ```FORMTYPE```.
Dans notre projet nous regardons uniquement le questionnaire "topical". 

In [4]:
df = dfs["2023"] # regardons d'abord l'année 2023
df.describe()

,HEIGHT,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,K2Q35A_1_YEARS,BIRTH_MO,BIRTH_YR,K6Q41R_STILL,...,HHCOUNT_IF,HIGRADE,HIGRADE_TVIS,FPL_I1,FPL_I2,FPL_I3,FPL_I4,FPL_I5,FPL_I6,FWC
count,32319.000000,55162.000000,55162.000000,54862.000000,55162.000000,55162.000000,1971.000000,54770.000000,54675.000000,18118.000000,...,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000
mean,151.687657,1.876690,1.455187,1.126135,8.339563,1.486422,4.868087,6.613255,2014.314092,1.884204,...,0.031761,2.829393,3.485606,287.280537,287.401998,287.531362,288.364091,287.652804,287.905080,1308.472653
std,20.522720,0.870582,0.787676,0.439835,5.303088,0.499820,3.478265,3.404391,5.324464,0.319990,...,0.175365,0.436074,0.799796,124.920459,124.878680,124.957976,124.475651,124.808909,124.524807,2334.642676
min,93.980000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2005.000000,1.000000,...,0.000000,1.000000,1.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,11.006690
25%,134.620000,1.000000,1.000000,1.000000,4.000000,1.000000,2.000000,4.000000,2009.000000,2.000000,...,0.000000,3.000000,3.000000,179.000000,179.000000,179.000000,182.000000,179.000000,181.000000,263.179394
50%,154.940000,2.000000,1.000000,1.000000,8.000000,1.000000,3.000000,7.000000,2015.000000,2.000000,...,0.000000,3.000000,4.000000,337.000000,337.000000,337.000000,337.500000,337.000000,337.000000,639.147322
75%,167.630000,2.000000,2.000000,1.000000,13.000000,2.000000,6.000000,10.000000,2019.000000,2.000000,...,0.000000,3.000000,4.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,1416.809570
max,211.050000,4.000000,4.000000,3.000000,17.000000,2.000000,15.000000,12.000000,2023.000000,2.000000,...,1.000000,3.000000,4.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,54078.402188


In [5]:
# La base de données de 2023 est dense, avec un total de 452 de variables sur 55162 individus (à travers les états-Unis).
# Sur l'ensemble des enfants dans l'échantillons : 
# (i) 21524 sont âgés de 0 à 5 ans,
# (ii) 18397 sont âgés de 6 à 11 ans et 
# (iii) 15241 sont âgés de 11 à 17 ans.

print(df["FORMTYPE"].value_counts())
print(df["FIPSST"].value_counts()[:5])

FORMTYPE
T1    21524
T3    18397
T2    15241
Name: count, dtype: int64
FIPSST
06    4696
20    2805
27    2400
22    2093
39    1735
Name: count, dtype: int64


On regarde le pourcentage de valeurs manquantes par colonne, par ordre décroissant.
On observe des taux de valeurs manquantes très élevés pour certaines questions qui ne sont posées que dans des cas très spécifiques et qui dépendent d’une réponse précédente. (Cela est rassurant.)

Par exemple, pour la variable ```CYSFIB_SCREEN``` (avec un taux de 99.94% de valeurs manquantes), celle-ci encode la question :

**Was this condition identified through a blood test done shortly after birth? … These tests are sometimes called newborn screening.** Cette question est ignorée si ```CYSTFIB = 2```. La variables ```CYSFIB``` a un taux très très faible (0.19%) de valeurs manquantes et parmi les valeurs prises, ```CYSFIB!=2``` dans seulement 30 questionnaires.

Pour les variables ```DIABETES_DESC``` et ```DIABETES_CURR```, les questions qu'elles encodent sont ignorées dès lorsque ```DIABETES = 2``` (ce qui regroupe seulement 40 questionnaires).

In [6]:
missing_values = df.isna().mean().sort_values(ascending=False) * 100

print("Valeurs manquantes en pourcentage par colonne\n",
    missing_values)
print("Nombre de colonnes avec au moins 50% de valeurs manquantes\n",(missing_values >= 50).sum())

# Illustration de l'exemple de CYSTFIB et DIABETES
print("---------------------------------")
print("Informations sur la variable CYSTFIB_SCREEN : ", guide[guide["Variable"] == "CYSTFIB_SCREEN"]["Universe"].iloc[0], "\n",
    guide[guide["Variable"] == "CYSTFIB"]["Question"].iloc[0])
print("Valeurs manquantes pour la variable CYSTFIB : \n",
    (df["CYSTFIB"].isna().mean() * 100).round(2), "%")
print(df["CYSTFIB"].value_counts())

print("---------------------------------")
print("Informations sur la variable DIABETES_CURR : ", guide[guide["Variable"] == "DIABETES_CURR"]["Universe"].iloc[0], "\n",
    guide[guide["Variable"] == "DIABETES"]["Question"].iloc[0])
print("Valeurs manquantes pour la variable DIABETES : \n",
    (df["DIABETES"].isna().mean() * 100).round(2), "%")
print(df["DIABETES"].value_counts())

Valeurs manquantes en pourcentage par colonne
 CYSTFIB_SCREEN    99.949240
DIABETES_DESC     99.945615
DIABETES_CURR     99.922048
K2Q38C            99.789710
LIVEUSA_MO        99.778833
                    ...    
FPL_I3             0.000000
FPL_I4             0.000000
FPL_I5             0.000000
FPL_I6             0.000000
FWC                0.000000
Length: 456, dtype: float64
Nombre de colonnes avec au moins 50% de valeurs manquantes
 180
---------------------------------
Informations sur la variable CYSTFIB_SCREEN :  Skip if CYSTFIB=2 
 Header: Has a doctor or other health care provider EVER told you that this child has......Cystic Fibrosis?
Valeurs manquantes pour la variable CYSTFIB : 
 0.19 %
CYSTFIB
2.0    55025
1.0       30
Name: count, dtype: int64
---------------------------------
Informations sur la variable DIABETES_CURR :  Skip if DIABETES=2 
 Header: Has a doctor or other health care provider EVER told you that this child has......Type 2 Diabetes?
Valeurs manquantes pou

### Choix des variables d'intérêt

Dans le guide, les variables sont classées dans plusieurs grandes catégories, en fonction de la question de recherche et également de la base de données associée. En vue de notre base de données, on peut garder uniquement les variables de type "Topical" et "Operational" (colonne "Source").

In [7]:
guide = guide[guide["Source"].isin({"Topical", "Operational"})]

years = {"2021", "2022", "2023", "2024"}

# On garde uniquement des variables qui sont communes aux quatre dataset NSCH
guide = guide[guide["Survey Years"].apply(
    lambda x: years <= set(y.strip() for y in str(x).split(","))
)]

# Cela réduit la base à 318 variables communes entre les databases
# Netoyer les dataframe NSCH avec cette contrainte, en rajoutant le code de l'état de résidance
# comme variable admissible.

variables_com = set(guide["Variable"])
variables_com = variables_com | {"FIPSST"}

for year in ["2023", "2022", "2021", "2024"]:
    df = dfs[year]
    df = df.loc[:, df.columns.isin(variables_com)]
    dfs[year] = df

In [8]:
counts = guide.groupby(["Source", "Topic"])["Variable"].count().reset_index(name="count")
print("Nombre de variable par groupe de (Source, Topic)\n",counts.to_string())

Nombre de variable par groupe de (Source, Topic)
          Source                            Topic  count
0   Operational                Data Quality Flag      1
1   Operational                        Geography      4
2   Operational                  Imputation Flag     10
3   Operational                      Operational      3
4   Operational                           Weight      3
5       Topical           A. This Child's Health     83
6       Topical       B. This Child as an Infant     19
7       Topical          C. Health Care Services     61
8       Topical         D. Health Care Providers     25
9       Topical     E. Health Insurance Coverage     13
10      Topical         F. Providing Health Care      8
11      Topical       G. Learning and Activities     33
12      Topical      H. About You and This Child     19
13      Topical     I. Your Family and Household     36
14      Topical  J. About You/Child's Caregivers     18
15      Topical         K. Household Information     1

In [9]:
# Sélectionner des variables dans l'univers "All Children" => cad des questions posées à toutes les groupes d'age

filter_variables = {
    var for var in variables_com
    if guide[guide["Variable"] == var]["Universe"].item() == "All Children"
}

# Proposition : 
final_variables = {

    # topical variables
    "ACE1", "ACE10", "ACE11", "ARRANGEHC", "ATHOMEHC", "AVOIDCHG", "C4Q04", "CURRCOV",
    "BLINDNESS", "BLOOD", "BREATHING", "CAVITIES", "DOWNSYN", "FOODSIT", "HCABILITY", "HEART", "HOWMUCH",
    "K2Q30A", "K2Q31A", "K2Q60A", "K4Q23", "K8Q11", "K8Q34", "SCREENTIME", "VIDEOPHONE", "SWALLOWING",
    
    # operational variables
    "BIRTH_YR_F", "FIPSST", "FORMTYPE",
    "METRO_YN", "MPC_YN", "STRATUM",
    "FWC", # child's weight
    "YEAR", "HHID"
}

cd.write_questions(final_variables, guide)

In [10]:
for year in ["2024", "2023", "2022", "2021"]:
    df = dfs[year]
    df = df.loc[:, df.columns.isin(final_variables)]
    dfs[year] = df

In [11]:
df = dfs["2024"]
df.describe()

,BREATHING,SWALLOWING,CAVITIES,BLINDNESS,HEART,DOWNSYN,BLOOD,K2Q60A,K2Q30A,K2Q31A,...,SCREENTIME,K8Q11,FOODSIT,HCABILITY,C4Q04,ACE1,METRO_YN,MPC_YN,BIRTH_YR_F,FWC
count,51176.000000,51041.000000,51175.000000,51177.000000,51275.000000,51195.000000,51152.000000,51164.000000,51151.000000,50998.000000,...,50507.000000,50451.000000,50122.000000,51051.000000,51062.000000,50461.000000,46525.000000,44237.000000,51375.000000,51375.000000
mean,1.924359,1.983993,1.895085,1.985697,1.973983,1.997558,1.994663,1.986377,1.918770,1.868348,...,3.230879,3.095241,1.339990,1.594837,1.286828,1.552565,1.176034,1.703506,0.053781,1411.265682
std,0.264425,0.125502,0.306446,0.118739,0.159186,0.049353,0.072861,0.115920,0.273191,0.338116,...,1.356668,0.889865,0.570331,0.875251,0.569268,0.764085,0.380854,0.456717,0.279815,2421.255016
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,12.335145
25%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,323.873253
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,723.611528
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,4.000000,4.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,0.000000,1522.186675
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,2.000000,2.000000,3.000000,54104.303178


In [12]:
missing_values = df.isna().mean().sort_values(ascending=False) * 100
print(missing_values)

MPC_YN        13.893917
METRO_YN       9.440389
ACE10          3.912409
ACE11          3.643796
VIDEOPHONE     3.283698
FOODSIT        2.438929
K8Q34          2.119708
K4Q23          1.866667
K8Q11          1.798540
ACE1           1.779075
SCREENTIME     1.689538
AVOIDCHG       1.290511
HOWMUCH        1.247689
ATHOMEHC       1.144526
ARRANGEHC      1.132847
K2Q31A         0.733820
CURRCOV        0.718248
SWALLOWING     0.650122
HCABILITY      0.630657
C4Q04          0.609246
K2Q30A         0.436010
BLOOD          0.434063
K2Q60A         0.410706
CAVITIES       0.389294
BREATHING      0.387348
BLINDNESS      0.385401
DOWNSYN        0.350365
HEART          0.194647
FIPSST         0.000000
STRATUM        0.000000
FORMTYPE       0.000000
BIRTH_YR_F     0.000000
FWC            0.000000
dtype: float64


In [13]:
for var in df.columns:
    print("Variable : " + var + " prend " + str(len(df[var].unique())) + " valeurs uniques")
    print(df[var].unique())

# On a une variable qui est à moitié catégorielles - le ID de l'état
# On a une variable continue - la taille (FWC) : 0 valeurs manquantes

Variable : FIPSST prend 51 valeurs uniques
['48' '47' '13' '39' '19' '31' '27' '40' '08' '21' '30' '09' '18' '12'
 '01' '20' '06' '22' '45' '54' '49' '16' '35' '56' '28' '34' '36' '42'
 '04' '38' '05' '46' '10' '15' '32' '33' '11' '51' '29' '17' '37' '02'
 '26' '55' '53' '41' '24' '23' '50' '25' '44']
Variable : STRATUM prend 2 valeurs uniques
['1' '2A']
Variable : FORMTYPE prend 3 valeurs uniques
['T3' 'T1' 'T2']
Variable : BREATHING prend 3 valeurs uniques
[ 2.  1. nan]
Variable : SWALLOWING prend 3 valeurs uniques
[ 2.  1. nan]
Variable : CAVITIES prend 3 valeurs uniques
[ 2.  1. nan]
Variable : BLINDNESS prend 3 valeurs uniques
[ 2.  1. nan]
Variable : HEART prend 3 valeurs uniques
[ 2.  1. nan]
Variable : DOWNSYN prend 3 valeurs uniques
[ 2.  1. nan]
Variable : BLOOD prend 3 valeurs uniques
[ 2.  1. nan]
Variable : K2Q60A prend 3 valeurs uniques
[ 2.  1. nan]
Variable : K2Q30A prend 3 valeurs uniques
[ 2.  1. nan]
Variable : K2Q31A prend 3 valeurs uniques
[ 1.  2. nan]
Variable : 

In [14]:
imputer = KNNImputer(n_neighbors=3, weights="uniform")

clean = df.drop(columns=["FIPSST", "FORMTYPE", "STRATUM"])

matrix = clean.to_numpy(dtype=int)
imputer.fit_transform(matrix)

/opt/python/lib/python3.13/site-packages/pandas/core/internals/managers.py:1772: RuntimeWarning: invalid value encountered in cast
  result[rl.indexer] = arr


array([[ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  1.41080000e+04],
       [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  7.99500000e+03],
       [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  1.59300000e+03],
       ...,
       [ 1.00000000e+00,  2.00000000e+00,  2.00000000e+00, ...,
        -9.22337204e+18,  0.00000000e+00,  1.72000000e+02],
       [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  1.14000000e+03],
       [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  2.53900000e+03]],
      shape=(51375, 30))